# Яндекс Музыка
Сравнение Москвы и Петербурга окружено мифами. Например:

Москва — мегаполис, подчинённый жёсткому ритму рабочей недели. Санкт-Петербург — культурная столица, со своими вкусами.
На данных Яндекс Музыки мы сравним поведение пользователей двух столиц.

### Цель исследования — проверить три гипотезы:

* Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
* В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
* Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

Исследование пройдёт в три этапа:

* Обзор данных.
* Предобработка данных.
* Проверка гипотез.

### Обзор данных
Составим первое представление о данных Яндекс Музыки.

In [1]:
# импорт библиотеки pandas
import pandas as pd 

In [2]:
# # чтение файла с данными и сохранение в df
try:
    df = pd.read_csv(r"D:\DOCS\datasets/yandex_music_project.csv")
except FileNotFoundError:
    df = pd.read_csv('/datasets/yandex_music_project.csv')

In [3]:
# вывел на экран первые десять строк таблицы:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получил общую информацию о таблице c помощью метода info():
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — object.

Согласно документации к данным:

* userID — идентификатор пользователя;
* Track — название трека;
* artist — имя исполнителя;
* genre — название жанра;
* City — город пользователя;
* time — время начала прослушивания;
* Day — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

#### Вывод

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### 2  Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

In [5]:
# вывел на экран названия столбцов:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с хорошим стилем:

* несколько слов в названии запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

In [6]:
# переименовываю столбцы
df = df.rename(columns={'  userID':'user_id','Track':'track','  City  ':'city','Day':'day'}) # переименование столбцов

In [7]:
# проверка результатов 
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [8]:
# посчитал, сколько в таблице пропущенных значений.
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в track и artist пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в genre могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. В данном проекте такой возможности нет. В связи с этим:

* заполним эти пропуски явными обозначениями;
* оценим, насколько они повредят расчётам.

In [9]:
# заменил пропущенные значения в столбцах track, artist и genre на строку 'unknown', 
# используя перебор названий столбцов в цикле
columns_to_replace = ['track','artist','genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown') 

In [10]:
# проверка результатов
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

In [11]:
# посчитал явные дубликаты в таблице:
print('Количество явных дубликатов:', df.duplicated().sum())

Количество явных дубликатов: 3826


In [12]:
# удалил явные дубликаты
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
# проверка резульатов
print('Количество явных дубликатов:', df.duplicated().sum())

Количество явных дубликатов: 0


Теперь избавимся от неявных дубликатов в колонке genre. Название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:

* извлечем нужный столбец датафрейма;
* применим к нему метод сортировки;
* для отсортированного столбца вызовим метод, который вернёт уникальные значения из столбца.

In [14]:
# просмотрел уникальные названий жанров
genre_sorted = df['genre'].sort_values().unique()
genre_sorted

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Обнаружены следующие неявные дубликаты:

* hip,
* hop,
* hip-hop.

In [15]:
# чтобы очистить от них таблицу использовал метод replace()
duplicates = ['hip','hop','hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, name)# Устранение неявных дубликатов

In [16]:
# проверил данные на неявные дубликаты
genre_sorted = df['genre'].sort_values().unique()
genre_sorted

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

### Выводы

Предобработка обнаружила три проблемы в данных:

* нарушения в стиле заголовков,
* пропущенные значения,
* дубликаты — явные и неявные.

Выполнено исправление заголовков, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения замении на 'unknown'. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке genre.

Теперь можно перейти к проверке гипотез.

### 3  Проверка гипотез

#### Сравнение поведения пользователей двух столиц
#### Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [17]:
# сгруппировал данные по городу и посчитал количество прослушиваний в каждой группе.
df.groupby('city')['time'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [18]:
# сгруппировал данные по дню недели и посчитал прослушивания в понедельник, среду и пятницу.
df.groupby('day',)['time'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Теперь напишем функцию, которая объединит два эти расчёта.

Создадим функцию number_tracks(), которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:

* день недели,
* название города.

В функции сохраним переменную строки исходной таблицы, у которых значение:

* в колонке day равно параметру day,
* в колонке city равно параметру city.

Для этого применим последовательную фильтрацию с логической индексацией.

Затем посчитаем значения в столбце user_id получившейся таблицы. Результат сохраним в новую переменную и вернем эту переменную из функции.

In [19]:
def number_tracks(day, city):
    track_list = df.loc[df.loc[:,'day'] == day]
    track_list = track_list.loc[track_list.loc[:,'city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [20]:
print('Количество прослушиваний в Москве по понедельникам:', number_tracks('Monday', 'Moscow'))

Количество прослушиваний в Москве по понедельникам: 15740


In [21]:
print('Количество прослушиваний в Санкт-Петербурге по понедельникам:', number_tracks('Monday', 'Saint-Petersburg'))

Количество прослушиваний в Санкт-Петербурге по понедельникам: 5614


In [22]:
print('Количество прослушиваний в Москве по средам:', number_tracks('Wednesday', 'Moscow'))

Количество прослушиваний в Москве по средам: 11056


In [23]:
print('Количество прослушиваний в Санкт-Петербурге по средам:', number_tracks('Wednesday', 'Saint-Petersburg'))

Количество прослушиваний в Санкт-Петербурге по средам: 7003


In [24]:
print('Количество прослушиваний в Москве по пятницам:', number_tracks('Friday', 'Moscow'))

Количество прослушиваний в Москве по пятницам: 15945


In [25]:
print('Количество прослушиваний в Санкт-Петербурге по пятницам:', number_tracks('Friday', 'Saint-Petersburg'))

Количество прослушиваний в Санкт-Петербурге по пятницам: 5895


Создадим таблицу, где

* названия колонок — ['city', 'monday', 'wednesday', 'friday'];
* данные — результаты, которые вы получили с помощью number_tracks

In [26]:
# таблица с результатами
data = [
    ['Moscow', '15740', '11056', '15945'],
    ['Saint-Petersburg', '5614', '7003', '5895']
]
columns = ['city', 'monday', 'wednesday', 'friday']
music_time = pd.DataFrame(data = data, columns = columns)
music_time

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


#### Выводы

Данные показывают разницу поведения пользователей:

* В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
* В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Данные говорят в пользу первой гипотезы.

#### Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:

по Москве — в moscow_general;
по Санкт-Петербургу — в spb_general.

In [27]:
# получил таблицу moscow_general
moscow_general = df[df['city'] == 'Moscow']
moscow_general

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [28]:
# получил таблицу spb_genera
spb_general = df[df['city'] == 'Saint-Petersburg'] 
spb_general

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
...,...,...,...,...,...,...,...
61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Saint-Petersburg,21:14:40,Monday
61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Saint-Petersburg,21:06:50,Monday
61241,29E04611,Bre Petrunko,Perunika Trio,world,Saint-Petersburg,13:56:00,Monday
61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Saint-Petersburg,09:22:13,Monday


Создадим функцию genre_weekday() с четырьмя параметрами:

* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm',
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [29]:
# Объявил функции genre_weekday() с параметрами table, day, time1, time2,
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставил в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day] 
    # оставил в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2]
    # оставил в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]
    # сгруппировал отфильтрованный датафрейм по столбцу с названиями жанров, 
    # взял столбец genre и посчитал кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортировал результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False) 
    # вернул Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]   

Cравним результаты функции genre_weekday() для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [30]:
# вызвал функцию genre_weekday для утра понедельника в Москве 
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [31]:
# вызвал функцию genre_weekday для утра понедельника в Петербурге 
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [32]:
# вызвал функцию genre_weekday для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [33]:
# вызвал функцию genre_weekday для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

#### Выводы

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

В Москве пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:

* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.
* Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

#### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [34]:
# cгруппировал таблицу moscow_general по жанру и посчитал прослушивания треков каждого жанра
moscow_genres = moscow_general.groupby('genre')['genre'].count()
# отсортировал результат в порядке убывания и сохранил его в таблице moscow_genres.
moscow_genres = moscow_genres.sort_values(ascending = False)

In [35]:
# вывел первые 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [36]:
# cгруппировал таблицу spb_general по жанру и посчитал прослушивания треков каждого жанра
spb_genres = spb_general.groupby('genre')['genre'].count()
# отсортировал результат в порядке убывания и сохранил его в таблице spb_genres.
spb_genres = spb_genres.sort_values(ascending = False)

In [37]:
# вывел первые 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

#### Вывод
Гипотеза частично подтвердилась:

* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

### 4  Итоги исследования
Было проверено три гипотезы и установлено:

День недели по-разному влияет на активность пользователей в Москве и Петербурге.
#### Первая гипотеза полностью подтвердилась.

Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

#### Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.
#### Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.